# FLDAS Explorer Dashboard
Modified: Jun 13, 2019

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
#import utils
import hv_utils as  hvu

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Vector Tile overlay creation helpers
Modified: Jun 8, 2019

In [ ]:
VECTILE_CACHE={}
from VectorTile import VectorTile

Quick example

In [ ]:
lat, lon = 48.8566101, 2.3514992
z = 15
vtile = VectorTile.from_latlonz(lat, lon, z)
vtile.info()

gdf = vtile.to_gdf()
# display(gdf.head())

ndoverlay = vtile.to_ndoverlay()
# display(ndoverlay)

Modified: Jun 16, 2019

In [ ]:
vtile.fetch_zooms()

## Set default holoviews style options

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover']),
    opts.RGB(active_tools=['wheel_zoom'], tools=['hover']),
    opts.Overlay(active_tools=['wheel_zoom']),
)


---
## Load Datasets

In [ ]:
# Southern Africa Dataset
fpath_sa = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
fpath_ea = '/home/hayley/data/mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

#set height, width for images (for plotting)
H_IMG, W_IMG = 400, 400

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile
Modified: Jun 13, 2019

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:

H,W = 500,500
# basemap = gvts.EsriImagery
# basemap
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'

basemap = gv.tile_sources.EsriImagery

In [ ]:
%%opts WMTS [width=W, height=H]

basemap * timg_ea

# gvts.EsriNatGeo*timg_ea

---
## Add PointXY linked stream
Modified: Jun 16, 2019

- Fetch the lat,lon location from the mouse location

In [ ]:
from holoviews.streams import Tap, Selection1D, PointerXY, RangeXY

In [ ]:
# point_src = timg_ea
# pointxy = PointerXY(name='pt_latlon', 
#                     x=0., y=0.,source=point_src)
# pointxy.print_param_values()
# selection = Selection1D(source=timg_ea)


In [ ]:
def get_tseries(xrd, varname, x, y, method='nearest', curve_opts={}):
    print(f'x,y: {x,y}')
    return hv.Curve(xrd[varname].sel(X=x, Y=y,method=method)).opts(**curve_opts)

In [ ]:
curve_opts = {'framewise':True}
varname = varnames[1]
# timg_ea +\
# hv.DynamicMap(lambda x,y: get_tseries(xrd_ea, varname, x, y,curve_opts=curve_opts),
#               streams=[pointxy])

---
## Add RangeXY linked stream
Modified: Jun 16, 2019

- Fetch the x and y ranges of the current view
    - Fetch appropriate vector tile

In [ ]:
range_src = timg_ea
x_range, y_range = hvu.lbrt2ranges(range_src.bounds.lbrt())

In [ ]:
rangexy = RangeXY(x_range = x_range,
                  y_range = y_range,
                  source=timg_ea)

In [ ]:
# rangexy.print_param_values()

In [ ]:
def cb_rangexy(x_range, y_range):
    lbrt = hvu.ranges2lbrt(x_range, y_range)
    print(f'x_range: {x_range}')
    print(f'y_range: {y_range}')
    print(f'lbrt: ', lbrt)
    
    df = pd.DataFrame( [lbrt], columns='min_x min_y max_x max_y'.split() )
    return hv.Table( df)


In [ ]:
# (
#     timg_ea + \
#     hv.DynamicMap(cb_rangexy, streams=[rangexy])
# ).cols(1)


---
## Add regional statistics computation
Modified: Jun 16, 2019


---
## Putting the two streams together
Modified: Jun 16, 2019

In [ ]:
curve_opts = {'framewise':True}
img_opts = dict(width=W, height=H,
                axiswise=True, framewise=False,
               tools=['hover'],
               active_tools=['wheel_zoom'])

varname = varnames[1]

# Tab stream
tap_latlon = Tap(name='tap_latlon', x = 0.0, y=0.0, source=timg_ea)
dmap_tab =  hv.DynamicMap(lambda x,y: get_tseries(xrd_ea, varname, x, y),
              streams=[tap_latlon])

# Range stream
dmap_range_tbl = hv.DynamicMap(cb_rangexy, streams=[rangexy])

# Putting all together
layout = timg_ea+ dmap_tab + dmap_range_tbl

layout.opts(
    opts.Image(**img_opts),
    opts.Curve(**curve_opts),
).cols(1)
       



---
## Panel Dashboards
Modified: Jun 10, 2019


In [ ]:
# import param
import panel as pn
pn.extension()
# from holoviews.streams import Params
from geoviews.tile_sources import EsriImagery
from colorcet import palette

In [ ]:
#dataset: hvd_ea, hvd_sa
xr_datasets = {
    'EA': xrd_ea,
    'SA': xrd_sa
}

hv_datasets = {
    'EA': hvd_ea,
    'SA': hvd_sa
}
april = pd.date_range('2019-04-01', periods=30)

In [ ]:
# create dmap 
xrd_ea_small = xrd_ea.isel(time=[3,4])
xrd_sa_small = xrd_sa.isel(time=[3,4])

## Wrong: this creates holomaps
# dmap_ea_small = hv.Dataset(xrd_ea_small, kdims).to(hv.Image, ['X','Y'], varname, label='EA')
# dmap_sa_small = hv.Dataset(xrd_sa_small, kdims).to(hv.Image, ['X','Y'], varname, label='SA')

# datashade opts
from colorcet import fire
dopts = dict(width=W, height=H,
#             x_sampling=0.5, 
#             y_sampling=0.5,
            )

img_opts = dict(width=W, height=H,
                axiswise=True, framewise=False,
               tools=['hover'],
               active_tools=['wheel_zoom'])


## FLDASExplorer with panel
Modified: Jun 13, 2019

In [ ]:
import datetime as dt

class FLDASExplorer(param.Parameterized):
    region = param.ObjectSelector(default='EA', objects=['EA', 'SA'])
    varname = param.ObjectSelector(default=varnames[0],objects=varnames)
    time = param.Date(dt.datetime(2019,4,1), bounds=(dt.datetime(2019, 4, 1), dt.datetime(2019, 4, 30)))
    alpha = param.Number(default=1.0, bounds=(0.0,1.0))
#     cmap = param.ObjectSelector(default='fire', objects=['fire'])

                                                    
    @param.depends('region', 'varname', 'time')#, 'alpha')#, 'cmap')
    def view(self):
        xrd = xrd_ea if self.region == 'EA' else xrd_sa
        img = gv.Image(xrd.sel(time=self.time)[self.varname], ['X','Y'], crs=ccrs.PlateCarree())    
        
    #     datashade returns a hv.DynamicMap which dynamically re-renders this img as we zoom/pan
        return basemap*datashade(img.opts(**img_opts), 
                                 cmap=fire,#self.cmap, 
#                                  alpha=self.alpha,
                                 **dopts)


In [ ]:
explorer = FLDASExplorer()
# img_dmap = hv.DynamicMap(explorer.view)
app = pn.Row( explorer.param, explorer.view)

In [ ]:
app.servable()

todo: 
    - add a time slider and link it to the time input box widget
    - handle projection better (for the basemap and the latlon- data)
    - change panel pane's width and height for the image layout
    - geo statistics overlay using raster(...) and datashader operations
   

In [ ]:
basemap